In [1]:
import torch
import torch.nn as nn
from numpy.random.mtrand import rand
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BertConfig, BertForSequenceClassification, BertTokenizer
from datasets import load_dataset
from tqdm import tqdm
import random
from torch.distributions import Laplace
import numpy as np
from copy import deepcopy
import torch.nn.functional as F
class FeatureHook:
    """Intercepte les activations d'une couche specifique."""
    def __init__(self, module):
        self.hook = module.register_forward_hook(self.hook_fn)
        self.features = None

    def hook_fn(self, module, input, output):
        # Pour les transformers, output peut etre un tuple
        if isinstance(output, tuple):
            self.features = output[0]
        else:
            self.features = output

    def close(self):
        self.hook.remove()
# --- Classe Principale Uchida pour Sentiment ---

class LaplacianSentiment:
    def __init__(self, model_id="prajjwal1/bert-tiny", device="cuda"):
        self.device = device
        self.model_id = model_id
        
        # Chargement du modele (bert-tiny: 4.4M params, ~15x plus rapide que DistilBERT)
        # self.model = AutoModelForSequenceClassification.from_pretrained(
        #     model_id, num_labels=2, use_safetensors=True
        # ).to(device)

        config = BertConfig.from_pretrained(model_id, num_labels=2)
        self.model = BertForSequenceClassification.from_pretrained(model_id, config=config).to(device)
        self.tokenizer=BertTokenizer.from_pretrained(model_id)
        # self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        
        # Configuration par defaut
        self.config = {
            # "layer_name": "classifier.weight",  # Couche de classification
             "layer_name": "bert.encoder.layer.1.output.dense.weight",
            "hook_layer":  "bert.encoder.layer.1.output.dense",

            "watermark_size": 64,
            "lr": 5e-5,  # Learning rate plus eleve pour petit modele
            "spreading_factor": 4,     # Facteur d'étalement (S)
            "variance": 1.0,

            "epochs": 30  # Plus d'epochs car modele non-pretrained sur sentiment
        }
        
        self.saved_keys = {}
        
    def list_layers(self):
        """Affiche les couches disponibles pour le watermarking."""
        print("Couches disponibles:")
        for name, param in self.model.named_parameters():
            print(f"  {name}: {param.shape}")

    def _get_target_layer(self, model, layer_name=None):
        """Recupere la couche cible pour le hook."""

        for name, module in model.named_modules():
            if name == layer_name:
                return module
        raise ValueError(f"Couche {layer_name} introuvable.")


    def _get_target_layer_weight(self, model):
        """Récupère le tenseur des poids de la couche cible."""
        for name, param in model.named_parameters():
            if name == self.config["layer_name"]:
                return param
        raise ValueError(f"Paramètre {self.config['layer_name']} introuvable.")


    @staticmethod
    def _get_rand_bits(size):
        """Génère une séquence binaire aléatoire."""
        return [random.randint(0, 1) for _ in range(size)]

    def keygen(self, watermark_size):
        """
        Génère le watermark et les signes correspondants.
        Returns:
            wat_signs: Watermark en {-1, +1}
            watermark: Watermark binaire en {0, 1}
        """
        watermark = torch.tensor(self._get_rand_bits(watermark_size)).float().to(self.device)
        wat_signs = torch.tensor([-1.0 if i == 0 else 1.0 for i in watermark], dtype=torch.float).to(self.device)
        return wat_signs, watermark

    def generate_laplacian_pseudo_sequence(self, s, l, loc=0.0, scale=1.0, seed=None):
        """
        Génère une séquence pseudo-aléatoire avec distribution Laplacienne.

        Args:
            s: Spreading factor
            l: Taille du watermark
            loc: Paramètre de localisation (moyenne)
            scale: Paramètre d'échelle (diversité)
            seed: Graine aléatoire pour la reproductibilité

        Returns:
            torch.Tensor: Séquence pseudo de taille (s * l,)
        """
        if seed is not None:
            torch.manual_seed(seed)
            np.random.seed(seed)

        total_size = s * l
        laplace_dist = Laplace(loc=torch.tensor(loc), scale=torch.tensor(scale))
        pseudo_seq = laplace_dist.sample((total_size,)).to(self.device)

        return pseudo_seq

    def generate_weight_sequence(self, watermark_signs, pseudo_seq, S):
        """
        Génère la séquence de poids avec le facteur d'étalement S.
        wm_j = u_i · s_j
        """
        l = len(watermark_signs)
        n = len(pseudo_seq)

        if n < l * S:
            raise ValueError(f"Spreading sequence length ({n}) must be >= l*S ({l * S})")

        weight_seq = torch.zeros(l * S, dtype=torch.float32).to(self.device)

        for i in range(1, l + 1):
            start_idx = (i - 1) * S
            end_idx = i * S

            for j in range(start_idx, end_idx):
                weight_seq[j] = watermark_signs[i - 1] * pseudo_seq[j]

        return weight_seq

    def replace_weights(self, model, weight_sequence, selected_indices=None):
        """
        Remplace les poids sélectionnés avec les valeurs du watermark.
        """
        n = len(weight_sequence)
        target_layer = self._get_target_layer_weight(model)

        weight_tensor = target_layer.data
        original_shape = weight_tensor.shape
        flat_weights = weight_tensor.flatten()
        total_weights = flat_weights.numel()

        if n > total_weights:
            raise ValueError(f"Sequence length {n} exceeds total weights {total_weights}")

        if selected_indices is None:
            selected_indices = random.sample(range(total_weights), n)

        for i, idx in enumerate(selected_indices):
            flat_weights[idx] = weight_sequence[i]

        target_layer.data = flat_weights.reshape(original_shape)
        return selected_indices

    def pick_weights_by_indices(self, model, selected_indices):
        """
        Extrait les poids sélectionnés d'une couche.
        """
        target_layer = self._get_target_layer_weight(model)
        flat_weights = target_layer.data.flatten()
        selected_weights = flat_weights[selected_indices]
        return selected_weights

    def calculate_watermark_bits(self, selected_weights, spreading_sequence, S):
        """
        Calcule les bits du watermark à partir des poids extraits.
        """
        watermark_length = len(spreading_sequence) // S

        if len(spreading_sequence) != watermark_length * S:
            raise ValueError(f"Spreading sequence length must be divisible by S")

        watermark_bits = torch.zeros(watermark_length, dtype=torch.float32).to(self.device)
        correlation_sums = torch.zeros(watermark_length, dtype=torch.float32).to(self.device)

        for i in range(1, watermark_length + 1):
            start_idx = (i - 1) * S
            end_idx = i * S

            s_slice = spreading_sequence[start_idx:end_idx]
            w_slice = selected_weights[start_idx:end_idx]

            correlation_sum = torch.sum(s_slice * w_slice)
            correlation_sums[i - 1] = correlation_sum

            bit_value = 1.0 if correlation_sum >= 0 else 0.0
            watermark_bits[i - 1] = bit_value

        return watermark_bits, correlation_sums

    @staticmethod
    def _compute_ber(extracted, target):
        """Calcule le Bit Error Rate."""
        return (extracted != target).float().mean().item()
    
    def embed(self, train_dataloader, eval_dataloader=None):
        """
        Incorpore la marque Uchida pendant le finetuning.
        """
        print(f"--- Demarrage Embedding Laplacian ({self.config['layer_name']}) ---")
        print(f"Watermark size: {self.config['watermark_size']} bits")
        print(f"Spreading factor: {self.config['spreading_factor']}")
        watermarked_model = self.model
        watermarked_model.train()


        # 1. Génération des clés
        watermark_signs, watermark = self.keygen(self.config["watermark_size"])

        # Calcul de gamma (scale parameter)
        gamma = self.config["variance"] / np.sqrt(2)

        # Génération de la séquence pseudo-aléatoire Laplacienne
        pseudo_seq = self.generate_laplacian_pseudo_sequence(
            s=self.config['spreading_factor'],
            l=self.config['watermark_size'],
            loc=0.0,
            scale=gamma
        )

        # Génération de la séquence de poids
        weight_seq = self.generate_weight_sequence(
            watermark_signs=watermark_signs,
            pseudo_seq=pseudo_seq,
            S=self.config['spreading_factor']
        )

        print(f"Weight sequence length: {len(weight_seq)}")

        # 2. Remplacement initial des poids
        selected_indices = self.replace_weights(watermarked_model, weight_seq)
        print(f"Selected {len(selected_indices)} weight positions")

        # 2. Optimiseur
        optimizer = torch.optim.AdamW(watermarked_model.parameters(), lr=self.config["lr"])
        ce_loss = nn.CrossEntropyLoss()
        bce_loss = nn.BCELoss()
        
        # 3. Boucle d'entrainement
        for epoch in range(self.config["epochs"]):
            pbar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{self.config['epochs']}")
            total_loss = 0
            correct = 0
            total = 0

            for batch in pbar:
                input_ids = batch["input_ids"].to(self.device)  #tokennized textes (numbers)
                attention_mask = batch["attention_mask"].to(self.device) #which tokens to pay attention to (1 for real tokens, 0 for padding)
                labels = batch["labels"].to(self.device) #correct answer for each sentence ( 0 = negative, 1 = positive)
                
                optimizer.zero_grad()
                
                # A. Forward pass (Task Loss)
                outputs = watermarked_model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                l_main = ce_loss(logits, labels)
                

                
                # Loss Totale
                l_total = l_main
                
                l_total.backward()
                optimizer.step()
                self.replace_weights(
                    model=watermarked_model,
                    weight_sequence=weight_seq,
                    selected_indices=selected_indices
                )

                # D. Calcul du BER
                extracted_weights = self.pick_weights_by_indices(watermarked_model, selected_indices)
                extracted_wm, _ = self.calculate_watermark_bits(
                    extracted_weights, pseudo_seq, self.config['spreading_factor']
                )


                # Metrics
                total_loss += l_main.item()
                preds = torch.argmax(logits, dim=-1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
                
                ber = self._compute_ber(extracted_wm, watermark)
                acc = 100. * correct / total
                pbar.set_postfix(L_Main=f"{l_main.item():.3f}",BER=f"{ber:.2f}", Acc=f"{acc:.1f}%")
            
            # Evaluation en fin d'epoch
            if eval_dataloader:
                eval_acc = self.evaluate(watermarked_model, eval_dataloader)
                print(f"Epoch {epoch+1} | Eval Accuracy: {eval_acc:.2f}%")
        
        # Sauvegarde des cles
        self.saved_keys = {
           "watermark": watermark,
            "watermark_signs": watermark_signs,
            "pseudo_seq": pseudo_seq,
            "weight_seq": weight_seq,
            "selected_indices": selected_indices,
            "watermarked_model": watermarked_model
        }
        
        final_ber = self._compute_ber(extracted_wm, watermark)
        print(f"\n--- Embedding termine | BER final: {final_ber:.2f} ---")
        return watermarked_model
    
    def evaluate(self, model, dataloader):
        """Evalue la precision du modele."""
        model.eval()
        correct = 0
        total = 0
        
        with torch.no_grad():
            for batch in dataloader:
                input_ids = batch["input_ids"].to(self.device)
                attention_mask = batch["attention_mask"].to(self.device)
                labels = batch["labels"].to(self.device)
                
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                preds = torch.argmax(outputs.logits, dim=-1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        
        model.train()
        return 100. * correct / total
    
    def extract(self, suspect_model=None):
        """
        Extrait la marque d'un modele suspect.
        """
        if suspect_model is None:
            suspect_model = self.saved_keys["watermarked_model"]
        
        watermark = self.saved_keys["watermark"]
        pseudo_seq = self.saved_keys["pseudo_seq"]
        selected_indices = self.saved_keys["selected_indices"]
        
       # 1. Extraction des poids
        try:
            extracted_weights = self.pick_weights_by_indices(suspect_model, selected_indices)
        except ValueError:
            print("⚠️ Couche cible introuvable dans le modèle suspect.")
            return 1.0, None

        # 2. Calcul du watermark extrait
        extracted_wm, correlation_sums = self.calculate_watermark_bits(
            extracted_weights, pseudo_seq, self.config['spreading_factor']
        )

        # 3. Calcul du BER
        ber = self._compute_ber(extracted_wm, watermark)
        
        print(f"BER Extrait : {ber:.2f}")
        return ber, extracted_wm
    


print("Classe LaplacianSentiment chargee (bert-tiny: 4.4M params)")

/home/latim/PycharmProjects/WatDNN/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Classe LaplacianSentiment chargee (bert-tiny: 4.4M params)


In [2]:
# --- Chargement des donnees SST-2 ---

# Charger le dataset SST-2 (sentiment binaire)
dataset = load_dataset("glue", "sst2")

# Initialiser le watermarker (charge le modele bert-tiny)
laplac = LaplacianSentiment(device="cuda" if torch.cuda.is_available() else "cpu")

# Afficher les couches disponibles
laplac.list_layers()

# Tokenization
def tokenize_function(examples):
    return laplac.tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True) #converts raw text sentences into input_ids and attention_mask
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# DataLoaders (batch_size=64 possible grace au petit modele)
train_dataloader = DataLoader(tokenized_dataset["train"], batch_size=64, shuffle=True)
eval_dataloader = DataLoader(tokenized_dataset["validation"], batch_size=64)

print(f"\nTrain samples: {len(tokenized_dataset['train'])}")
print(f"Eval samples: {len(tokenized_dataset['validation'])}")

Loading weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 1862.90it/s, Materializing param=bert.pooler.dense.weight]
BertForSequenceClassification LOAD REPORT from: prajjwal1/bert-tiny
Key                                        | Status     | 
-------------------------------------------+------------+-
bert.embeddings.position_ids               | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.decoder.weight    

Couches disponibles:
  bert.embeddings.word_embeddings.weight: torch.Size([30522, 128])
  bert.embeddings.position_embeddings.weight: torch.Size([512, 128])
  bert.embeddings.token_type_embeddings.weight: torch.Size([2, 128])
  bert.embeddings.LayerNorm.weight: torch.Size([128])
  bert.embeddings.LayerNorm.bias: torch.Size([128])
  bert.encoder.layer.0.attention.self.query.weight: torch.Size([128, 128])
  bert.encoder.layer.0.attention.self.query.bias: torch.Size([128])
  bert.encoder.layer.0.attention.self.key.weight: torch.Size([128, 128])
  bert.encoder.layer.0.attention.self.key.bias: torch.Size([128])
  bert.encoder.layer.0.attention.self.value.weight: torch.Size([128, 128])
  bert.encoder.layer.0.attention.self.value.bias: torch.Size([128])
  bert.encoder.layer.0.attention.output.dense.weight: torch.Size([128, 128])
  bert.encoder.layer.0.attention.output.dense.bias: torch.Size([128])
  bert.encoder.layer.0.attention.output.LayerNorm.weight: torch.Size([128])
  bert.encoder.layer

Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1821/1821 [00:00<00:00, 34523.60 examples/s]


Train samples: 67349
Eval samples: 872


In [3]:

# Lancer l'embedding
watermarked_model = laplac.embed(train_dataloader, eval_dataloader)

--- Demarrage Embedding Laplacian (bert.encoder.layer.1.output.dense.weight) ---
Watermark size: 64 bits
Spreading factor: 4
Weight sequence length: 256
Selected 256 weight positions


Epoch 1/30: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:06<00:00, 154.31it/s, Acc=80.1%, BER=0.00, L_Main=0.519]


Epoch 1 | Eval Accuracy: 80.85%


Epoch 2/30: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:06<00:00, 161.41it/s, Acc=88.5%, BER=0.00, L_Main=0.210]


Epoch 2 | Eval Accuracy: 82.57%


...

Epoch 28 | Eval Accuracy: 78.78%


Epoch 29/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:07<00:00, 147.61it/s, Acc=98.7%, BER=0.00, L_Main=0.123]


Epoch 29 | Eval Accuracy: 78.44%


Epoch 30/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:06<00:00, 151.86it/s, Acc=98.8%, BER=0.00, L_Main=0.002]


Epoch 30 | Eval Accuracy: 78.78%

--- Embedding termine | BER final: 0.00 ---


In [4]:
# --- Test d'Extraction ---

print("=" * 50)
print("TEST D'EXTRACTION")
print("=" * 50)

# Extraction sur le modele watermarke
print("\n1. Extraction sur le modele watermarke:")
ber_watermarked, _ = laplac.extract(watermarked_model)

# Extraction sur un modele vierge (non-watermarke)
print("\n2. Extraction sur un modele vierge (devrait avoir BER ~0.5):")
# fresh_model = AutoModelForSequenceClassification.from_pretrained(
#     "prajjwal1/bert-tiny", num_labels=2, use_safetensors=True
# ).to(uchida.device)
model_id= "prajjwal1/bert-tiny"
config = BertConfig.from_pretrained(model_id, num_labels=2)
fresh_model = BertForSequenceClassification.from_pretrained(model_id, config=config).to(device="cuda" if torch.cuda.is_available() else "cpu")
tokenizer=BertTokenizer.from_pretrained(model_id)

ber_fresh, _ = laplac.extract(fresh_model)

print(f"\nResume:")
print(f"  - BER modele watermarke: {ber_watermarked:.2f} (attendu: ~0.0)")
print(f"  - BER modele vierge: {ber_fresh:.2f} (attendu: ~0.5)")

TEST D'EXTRACTION

1. Extraction sur le modele watermarke:
BER Extrait : 0.00

2. Extraction sur un modele vierge (devrait avoir BER ~0.5):


Loading weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 1549.87it/s, Materializing param=bert.pooler.dense.weight]
BertForSequenceClassification LOAD REPORT from: prajjwal1/bert-tiny
Key                                        | Status     | 
-------------------------------------------+------------+-
bert.embeddings.position_ids               | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.decoder.weight    

BER Extrait : 0.44

Resume:
  - BER modele watermarke: 0.00 (attendu: ~0.0)
  - BER modele vierge: 0.44 (attendu: ~0.5)


In [7]:
# --- Fonction de Distillation (Attaque) ---

def run_distillation_attack(laplac_obj, train_dataloader, eval_dataloader, epochs=5, lr=5e-5):
    """
    Tente de transferer la fonctionnalite du modele watermarke vers un modele vierge.
    Verifie si la marque (basee sur les poids) survit.
    """
    device = laplac_obj.device
    
    # 1. Teacher (Gele)
    teacher_model = laplac_obj.saved_keys["watermarked_model"]
    teacher_model.eval()
    for p in teacher_model.parameters():
        p.requires_grad = False
    
    # 2. Student (Vierge - Meme architecture)
    config = BertConfig.from_pretrained(model_id, num_labels=2)
    print("\n--- Initialisation du Student ---")
    student_model = BertForSequenceClassification.from_pretrained(model_id, config=config).to(device)
    student_model.train()
    
    # Verification initiale
    teacher_ber, _ = laplac_obj.extract(teacher_model)
    student_ber, _ = laplac_obj.extract(student_model)
    print(f"[Check] BER Teacher: {teacher_ber:.2f}")
    print(f"[Check] BER Student (Avant): {student_ber:.2f}")
    
    optimizer = torch.optim.AdamW(student_model.parameters(), lr=lr)
    history = {"loss": [], "ber": [], "acc": []}
    
    print(f"\n--- Distillation ({epochs} epochs) ---")
    feature_criterion = nn.MSELoss()
    for epoch in range(epochs):
        pbar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")
        running_loss = 0.0
        
        for batch in pbar:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            
            optimizer.zero_grad()

            teacher_layer = laplac._get_target_layer(teacher_model, laplac.config["hook_layer"])
            hook = FeatureHook(teacher_layer)
            
            # Output Matching (Soft Labels)
            with torch.no_grad():
                teacher_logits = teacher_model(input_ids=input_ids, attention_mask=attention_mask).logits
                teacher_probs = F.softmax(teacher_logits / 2.0, dim=-1)  # Temperature=2
                t_features = hook.features
            hook.close()

            student_layer = laplac._get_target_layer(student_model, laplac.config["hook_layer"])
            hook = FeatureHook(student_layer)
            student_logits = student_model(input_ids=input_ids, attention_mask=attention_mask).logits
            student_log_probs = F.log_softmax(student_logits / 2.0, dim=-1)
            s_features = hook.features
            hook.close()


            loss_feat = feature_criterion(s_features.mean(dim=[0, 1]), t_features.mean(dim=[0, 1])) + feature_criterion(s_features.var(dim=[0, 1]), t_features.var(dim=[0, 1]))


            # KL Divergence Loss
            loss_kl = F.kl_div(student_log_probs, teacher_probs, reduction="batchmean") * (2.0 ** 2)
            loss= loss_kl +loss_feat
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            pbar.set_postfix(Loss=f"{loss.item():.4f}")
        
        # Evaluation
        avg_loss = running_loss / len(train_dataloader)
        history["loss"].append(avg_loss)
        
        current_ber, pred_wm_prob = laplac_obj.extract(student_model)
        history["ber"].append(current_ber)
        
        eval_acc = laplac_obj.evaluate(student_model, eval_dataloader)
        history["acc"].append(eval_acc)
        
        print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f} | BER: {current_ber:.2f} | Acc: {eval_acc:.1f}% err_wat: {nn.BCELoss()(pred_wm_prob, laplac_obj.saved_keys["watermark"]).item()}")
    
    return student_model, history

In [8]:
# --- Lancer l'attaque par distillation ---

student_model, attack_history = run_distillation_attack(
    laplac,
    train_dataloader, 
    eval_dataloader,
    epochs= 1000, # Plus d'epochs possible car modele rapide
)


--- Initialisation du Student ---


Loading weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 2067.65it/s, Materializing param=bert.pooler.dense.weight]
BertForSequenceClassification LOAD REPORT from: prajjwal1/bert-tiny
Key                                        | Status     | 
-------------------------------------------+------------+-
bert.embeddings.position_ids               | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.decoder.weight    

BER Extrait : 0.00
BER Extrait : 0.44
[Check] BER Teacher: 0.00
[Check] BER Student (Avant): 0.44

--- Distillation (1000 epochs) ---


Epoch 1: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:05<00:00, 180.35it/s, Loss=1.0998]


BER Extrait : 0.41
Epoch 1 | Loss: 1.4984 | BER: 0.41 | Acc: 81.2% err_wat: 40.625


...

BER Extrait : 0.08
Epoch 642 | Loss: 0.0076 | BER: 0.08 | Acc: 77.5% err_wat: 7.8125


Epoch 643: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:06<00:00, 173.35it/s, Loss=0.0081]


BER Extrait : 0.08
Epoch 643 | Loss: 0.0069 | BER: 0.08 | Acc: 78.3% err_wat: 7.8125


Epoch 644:  86%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                  | 902/1053 [00:05<00:00, 166.62it/s, Loss=0.0038]


KeyboardInterrupt: 

In [ ]:
# --- Visualisation des Resultats ---

import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Loss
axes[0].plot(attack_history["loss"], 'b-o')
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Distillation Loss")
axes[0].set_title("Loss durant Distillation")
axes[0].grid(True)

# BER
axes[1].plot(attack_history["ber"], 'r-o')
axes[1].axhline(y=0.5, color='gray', linestyle='--', label='Random (0.5)')
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("BER")
axes[1].set_title("BER du Student (Watermark)")
axes[1].set_ylim([0, 1])
axes[1].legend()
axes[1].grid(True)

# Accuracy
axes[2].plot(attack_history["acc"], 'g-o')
axes[2].set_xlabel("Epoch")
axes[2].set_ylabel("Accuracy (%)")
axes[2].set_title("Accuracy du Student")
axes[2].grid(True)

plt.tight_layout()
plt.show()

# Resume final
print("\n" + "=" * 50)
print("RESUME FINAL")
print("=" * 50)
print(f"BER Teacher (watermarke): {laplac.extract(laplac.saved_keys['watermarked_model'])[0]:.2f}")
print(f"BER Student (apres distillation): {attack_history['ber'][-1]:.2f}")
print(f"Accuracy Student: {attack_history['acc'][-1]:.1f}%")

if attack_history['ber'][-1] > 0.4:
    print("\n-> La marque Uchida n'a PAS survecu a la distillation (BER eleve)")
else:
    print("\n-> La marque Uchida a survecu a la distillation (BER faible)")